# Image classification model customization guide

This python notebook will help you retrain a model which will be used later in our CS-4833 class.  
There are some optional documents you can read:

* [MediaPipe website](https://developers.google.com/mediapipe)
* [Image classification guide for Python](https://developers.google.com/mediapipe/solutions/vision/image_classifier/python)
* [Image classification for raspberry pi GitHub](https://github.com/googlesamples/mediapipe/tree/main/examples/image_classification/raspberry_pi)

In [ ]:
#@title License information
# Copyright 2023 The MediaPipe Authors.
# Licensed under the Apache License, Version 2.0 (the "License");
#
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

The MediaPipe Model Maker package is a low-code solution for customizing on-device machine learning (ML) Models.

The MediaPipe image classification solution provides several models you can use immediately for machine learning (ML) in your application. However, if you need to classify images with content not covered by the provided models, you can customize any of the provided models with your own data and MediaPipe [Model Maker](https://developers.google.com/mediapipe/solutions/model_maker). This model modification tool rebuilds a portion of the model using data you provide. This method is faster than training a new model and can produce a model that is more useful for your specific application.

The following sections show you how to use Model Maker to retrain a pre-built model for image classification with your own data, which you can then use with the MediaPipe [Image Classifier](https://developers.google.com/mediapipe/solutions/vision/image_classifier). The example retrains a general purpose classification model to classify images of flowers.

This notebook shows the end-to-end process of customizing an ImageNet pretrained image classification model for recognizing flowers defined in a user customized flower dataset.

## Setup

This section describes key steps for setting up your development environment to retrain a model. These instructions describe how to update a model using [Google Colab](https://colab.research.google.com/), and you can also use Python in your own development environment. For general information on setting up your development environment for using MediaPipe, including platform version requirements, see the [Setup guide for Python](https://developers.google.com/mediapipe/solutions/setup_python).

To install the libraries for customizing a model, run the following commands:

In [ ]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker

Use the following code to import the required Python classes:

In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import image_classifier

import matplotlib.pyplot as plt
%matplotlib inline

## Prepare data

Retraining a model for image classification requires a dataset that includes all kinds of items, or *classes*, that you want the completed model to be able to identify. You can do this by trimming down a public dataset to only the classes that are relevant to your usecase, compiling your own data, or some combination of both. The dataset can be significantly smaller that what would be required to train a new model.

### Upload our dataset

* Step 1: Organize our own dataset by folloing this pattern: `<image_path>/<label_name>/<image_names>.*`. The directory contains several subdirectories, each corresponding to specific class labels.
* Step 2: Compress your dataset folder to a zip file
* Step 3: Upload your zip file to this colab runtime
* Step 4: Uncompress it using the following code
    `!unzip dataset.zip`

In [ ]:
# uncompress the zip file, change the name if you use a different name
!unzip -o dataset.zip

In [ ]:
# specify our image_path to the folder name, change the name if you use a different name
image_path = "dataset"

### Review data

When preparing data for training with Model Maker, you should review the training data to make sure it is in the proper format, correctly classified, and organized in directories corresponding to classification labels. *This step is optional, but recommended.*

The following code block retrieves all the label names from the expected directory structure at `image_path` and prints them.

In [ ]:
print(image_path)
labels = []
for i in os.listdir(image_path):
    if os.path.isdir(os.path.join(image_path, i)):
        labels.append(i)
print(labels)

You can review a few of the example images from each category using the following code:

In [ ]:
NUM_EXAMPLES = 2

for label in labels:
  label_dir = os.path.join(image_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(4,3))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

### Create dataset

Training data for machine learning can be large, consisting of hundreds or thousands of files which typically do not fit into available memory. You must also split it into groups for different uses: training, testing, and validation. For these reasons, Model Maker uses a `Dataset` class to organize training data and feed it to the retraining process.

To create a dataset, use the `Dataset.from_folder` method to load the data located at `image_path` and split it into training, testing, and validation groups:

In [ ]:
data = image_classifier.Dataset.from_folder(image_path)
train_data, remaining_data = data.split(0.8)
test_data, validation_data = remaining_data.split(0.5)

In this example, 80% of the data is used for training, with the remaining data split in half, so that 10% of the total is used for testing, and 10% for validation.

## Retrain model

Once you have completed preparing your data, you can begin retraining a model to build a new classification layer that can recognize the items types, or classes, defined by your training data. This type of model modification is called [transfer learning](https://www.wikipedia.org/wiki/Transfer_learning). The instructions below use the data prepared in the previous section to retrain an image classification model to recognize different types of flowers.

**Note:** For this type of model, the retraining process causes the model to forget any classes it was previously able to recognize. Once the retraining is complete, the new model can *only* recognize classes trained from the new dataset.

### Set retraining options

There are a few required settings to run a retraining aside from your training dataset: output directory for the model and the model architecture. Use `HParams` object `export_dir` parameter to specify a model output directory. Use the `SupportedModels` class to specify the model architecture. The image classifier solution supports the following model architectures:

- `MobileNet-V2`
- `EfficientNet-Lite0`
- `EfficientNet-Lite2`
- `EfficientNet-Lite4`

To set the required parameters, use the following code:

In [ ]:
spec = image_classifier.SupportedModels.MOBILENET_V2
hparams = image_classifier.HParams(export_dir="exported_model", epochs=10)
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)

This example code uses MobileNetV2 model architecture, which you can learn more about from the  [MobileNetV2](https://arxiv.org/abs/1801.04381) research paper. The retraining process has many additional options, however most of them are set for you automatically. You can learn about these optional parameters in the [Retraining parameters](#retraining_parameters) section.

### Run retraining

With your training dataset and retraining options prepared, you are ready to start the retraining process. This process is resource intensive and can take a few minutes to a few hours depending on your available compute resources. Using a Google Colab environment with standard CPU processing, the example retraining below takes about 20 minutes to train on approximately 4000 images. You can typically decrease your training time by using GPU processors.

To begin the retraining process, use the `create()` method with dataset and options you previously defined:

In [ ]:
model = image_classifier.ImageClassifier.create(
    train_data = train_data,
    validation_data = validation_data,
    options=options,

)

### Evaluate performance

After retraining the model, you should evaluate it on a test dataset, which is typically a portion of your original dataset not used during training. Accuracy levels between 0.8 and 0.9 are generally considered very good, but your use case requirements may differ. You should also consider how fast the model can produce an inference. Higher accuracy frequently comes at the cost of longer inference times.

To run an evaluation of the example model, run it against the test portion of the dataset:

In [ ]:
loss, acc = model.evaluate(test_data)
print(f'Test loss:{loss}, Test accuracy:{acc}')

**Caution:** While high accuracy of a model is a common goal for machine learning models, you should be cautious of training to a point of [overfitting](https://en.wikipedia.org/wiki/Overfitting), which causes the model to perform extremely well with its training data, but quite poorly on new data.

## Export model

After retraining a model, you must export it to Tensorflow Lite model format to use it with the MediaPipe in your application. The export process generates required model metadata, as well as a classification label file.

To export the retrained model for use in your application, use the following command:

In [ ]:
model.export_model()

Use the follow commands with Google Colab to list model and download it to your development environment:

In [ ]:
!ls exported_model
files.download('exported_model/model.tflite')

## Performance benchmarks

Below is a summary of our benchmarking results for the supported model architectures. These models were trained and evaluated on the same flowers dataset as this notebook. When considering the model benchmarking results, there are a few important caveats to keep in mind:
- The test accuracy column reflects models which were trained with the default parameters. To optimize model performance, experiment with different model and retraining parameters in order to obtain the highest test accuracy. Refer to the [Retraining parameters](#retraining_parameters) section for more information on customizing these settings.
- The larger model architectures, such as EfficientNet_Lite4, may not acheive the highest test accuracy on simpler datasets like the flowers dataset used this notebook. Research suggests that these larger model architecture can outperform the others on more complex datasets like ImageNet, for more information, see [EfficientNet paper](https://arxiv.org/pdf/1905.11946.pdf). The ImageNet dataset is more complex, with over a million training images and 1000 classes, while the flowers dataset has only 3670 training images and 5 classses.

<table>
<thead>
<tr>
<th>Model architecture</th>
<th>Test Accuracy</th>
<th>Model Size</th>
<th>CPU 1 Thread Latency(Pixel 6)</th>
<th>GPU Latency(Pixel 6)</th>
<th>EdgeTPULatency(Pixel 6)</th>
</tr>
</thead>
<tbody>
<tr>
<td>MobileNet_V2</td>
<td>85.4%</td>
<td><strong>8.9MB</strong></td>
<td>29.12</td>
<td>77.77</td>
<td>31.14</td>
</tr>
<tr>
<td>EfficientNet_Lite0</td>
<td>91.3%</td>
<td>13.5MB</td>
<td><strong>15.6</strong></td>
<td><strong>9.25</strong></td>
<td><strong>16.72</strong></td>
</tr>
<tr>
<td>EfficientNet_Lite2</td>
<td><strong>91.5%</strong></td>
<td>19.2MB</td>
<td>35.2</td>
<td>13.94</td>
<td>37.52</td>
</tr>
<tr>
<td>EfficientNet_Lite4</td>
<td>90.8%</td>
<td>46.8MB</td>
<td>103.16</td>
<td>23.14</td>
<td>114.67</td>
</tr>
</tbody>
</table>